Pada sub-modul ini, kita akan belajar menjalankan model yang telah kita buat untuk melakukan tugas prediksi dalam lingkup klasifikasi gambar. Tugas prediksi akan dijalankan di android menggunakan TensorFlow Lite. Adapun pengetahuan dasar terkait Android Developer dapat membantu Anda untuk  memahami sub-modul ini dengan baik.

Berikut beberapa hal yang akan kita pelajari:

* Memuat dan menjalankan model di Android dengan TensorFlow Lite.
* Menguji hasil deployment melalui proses klasifikasi input gambar yang diberikan.

Pada latihan ini, kita akan membuat aplikasi Android yang dapat mengklasifikasi input gambar. Model yang digunakan akan dilatih menggunakan dataset dari OpenImages https://storage.googleapis.com/openimages/web/index.html. OpenImages menyediakan banyak dataset gambar yang bisa kita manfaatkan.

Anda dapat mengunduh contoh proyek pada tautan berikut: contoh proyek. 

Tugas Anda adalah melengkapi bagian yang kosong sesuai instruksi yang diberikan.



# **Kebutuhan Sistem**
Latihan kali ini adalah menjalankan Aplikasi Android menggunakan Android Emulator.  Perangkat lunak yang dibutuhkan, antara lain:

* IDE Android Studio.
* Android Emulator

Anda dapat membuat Android Emulator melalui Android Studio dengan langkah-langkah berikut:

1. Buka AVD Manager (Tools > AVD Manager)
2. Tekan Create Virtual Device, pilih hardware profile yang anda inginkan (pada latihan ini menggunakan “Pixel 5”) lalu tekan Next
3. Pilih system image yang diinginkan (pada latihan ini menggunakan “R”) lalu tekan Next
4. Tekan Finish

Langkah-langkah di atas bila ditunjukkan dalam bentuk gambar akan terlihat seperti berikut.

https://dicoding-web-img.sgp1.cdn.digitaloceanspaces.com/original/academy/dos:f4c2c68f7b40aafe36904b600162f84820211208154638.jpeg

Untuk menjalankan emulator, tekanlah tombol run pada AVD Manager. Berikut contoh tampilan dari Android Emulator.

https://dicoding-web-img.sgp1.cdn.digitaloceanspaces.com/original/academy/dos:daabca667151ca5108c02add3cb83ebb20211208154638.jpeg



# **Penjelasan Proyek**
Tujuan dari proyek ini adalah membuat sebuah model yang dapat mengklasifikasi gambar bahan makanan. Model ini nantinya akan dimuat dan dijalankan pada sebuah aplikasi Android sederhana menggunakan TensorFlow Lite.

Proyek ini menggunakan dataset gambar bahan makanan yang diperoleh dari OpenImage. Dataset yang digunakan terdiri dari lima kategori bahan makanan yaitu Broccoli, Carrot, Cheese, Potato, dan Tomato.

Berikut merupakan tampilan awal dari aplikasi Android sederhana yang akan kita buat pada proyek ini.

https://dicoding-web-img.sgp1.cdn.digitaloceanspaces.com/original/academy/dos:c00b10f6329422604eb837425388668520211208155400.png

# **Penjelasan File**
Setelah mengunduh contoh proyek, Anda akan menjumpai beberapa file dan folder sebagai berikut:

* ‘dataset_1’ merupakan sebuah folder berisi dataset untuk melatih dan mengevaluasi model.
* `notebook.ipynb` merupakan sebuah berkas notebook yang dapat dijalankan melalui Google Colab. Notebook ini berisi tahapan dalam membuat dan mengonversi model.
* `saved_model` merupakan sebuah folder yang berisi model yang disimpan dalam format SavedModel yang selanjutnya akan dikonversi menjadi tflite model.
* `vegs.tflite` merupakan model yang telah dikonversi menjadi tflite model. Model ini digunakan untuk menerapkan model klasifikasi gambar yang dijalankan di aplikasi Android.
* `vegs.txt` merupakan sebuah berkas text yang berisi label untuk dataset.
* `SimpleImageClassification` merupakan folder project Android Studio yang digunakan untuk membuat aplikasi Android.

Berikut merupakan tree structure dari contoh proyek ini:

In [ ]:
├───dataset_1
│   ├───test
│   │   ├───Broccoli
│   │   ├───Carrot
│   │   ├───Cheese
│   │   ├───Potato
│   │   └───Tomato
│   └───train
│       ├───Broccoli
│       ├───Carrot
│       ├───Cheese
│       ├───Potato
│       └───Tomato
├───images
├───saved_model
├───SimpleImageClassification    
    ├───app
    │   ├───build
    │   └───src
    │       ├───androidTest
    │       ├───main
    │           ├───assets
    │           ├───java
    │           └───res
    └───gradle

# **Penjelasan Notebook**
Di sini kita akan membahas tahap pembuatan model untuk mengklasifikasi gambar bahan makanan. Sebelum membuat model, kita perlu mempersiapkan semua library yang dibutuhkan.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input
import numpy as np
import matplotlib.pyplot as plt
import pathlib

Tahap berikutnya adalah mempersiapkan dataset yang akan digunakan. Kemudian, lakukan tahap preprocessing data berikut:

* Melakukan image augmentation.
* Mempersiapkan data latih dan data evaluasi.
* Memberi label pada dataset secara otomatis.

In [ ]:
TRAINING_DIR = "dataset_1/train"
VALIDATION_DIR = "dataset_1/test"
 
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    vertical_flip=True,
    fill_mode='nearest'
)
 
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    class_mode='categorical',
                                                    target_size=(150, 150))
 
validation_datagen = ImageDataGenerator(
    rescale=1.0/255
)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                        class_mode='categorical',
                                                        target_size=(150, 150))

Selanjutnya, buat dan latihlah model menggunakan dataset yang telah kita siapkan sebelumnya. Teknik yang akan digunakan kali ini adalah transfer learning memanfaatkan model `MobileNetV2`. https://www.tensorflow.org/api_docs/python/tf/keras/applications/mobilenet_v2/MobileNetV2

In [ ]:
pre_trained_model = MobileNetV2(weights="imagenet", include_top=False,
                                input_tensor=Input(shape=(150, 150, 3)))
 
for layer in pre_trained_model.layers:
    layer.trainable = False
 
last_output = pre_trained_model.output
 
x = tf.keras.layers.Flatten(name="flatten")(last_output)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dense(5, activation='softmax')(x)
 
model = tf.keras.models.Model(pre_trained_model.input, x)
 
int_lr = 1e-4
num_epochs = 30
 
optimizer = tf.optimizers.Adam(lr=int_lr)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
 
H = model.fit(train_generator,
              epochs=num_epochs,
              validation_data=validation_generator)

Setelah performa model dirasa cukup, kita dapat menyimpan model tersebut dalam format SavedModel. Setelah itu, ubah SavedModel menjadi tflite model menggunakan perintah berikut:

In [ ]:
# Menyimpan model dalam format SavedModel
export_dir = 'saved_model/'
tf.saved_model.save(model, export_dir)
 
# Convert SavedModel menjadi vegs.tflite
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()
 
tflite_model_file = pathlib.Path('vegs.tflite')
tflite_model_file.write_bytes(tflite_model)

Jika semua tahapan berjalan dengan baik, akan muncul sebuah file model baru bernama `vegs.tflite`.

# **Menerapkan Model pada Aplikasi Android dengan TensorFlow Lite**
Sebelum menerapkan model tflite pada aplikasi Android, buatlah file `vegs.txt` yang berisi label dari dataset yang kita gunakan. Caranya mudah, bukalah aplikasi Notepad di komputer Anda dan sailnlah label-label berikut. 

In [ ]:
Broccoli
Carrot
Cheese
Potato
Tomato

Lalu, simpan berkas tersebut dengan nama ‘vegs.txt’.

Untuk menerapkan model tflite pada aplikasi Android, buatlah sebuah proyek sederhana di Android Studio. Anda akan menemukan sebuah folder “SimpleImageClassification” yang merupakan contoh proyek sederhana di Android Studio. Buka folder tersebut menggunakan Android Studio dengan mengikuti petunjuk di bawah ini.



**Membuat folder ‘assets’**

Salin file `vegs.tflite` dan file `vegs.txt` pada folder assets yang terdapat dalam path app > src > main. Tahap ini dilakukan agar Android Studio dapat secara otomatis memanggil file model beserta labelnya.



**Membuat file ‘TFLiteHelper.java’**

Masih di proyek yang sama, selanjutnya bukalah berkas ‘TFLiteHelper.java’. Berkas tersebut  terletak dalam path app > src > main > java > com > example > imageclassification. Kemudian di dalam TFLiteHelper.java, lakukan inisiasi Interpreter TensorFlow Lite dengan kode berikut.

In [ ]:
  void init() {
        try {
            Interpreter.Options opt = new Interpreter.Options();
            tflite = new Interpreter(loadmodelfile(context), opt);
        } catch (Exception e) {
            e.printStackTrace();
        }
    }

Tahap berikutnya adalah melakukan preprocessing terhadap input gambar yang dimasukkan oleh pengguna. Pada tahap preprocessing, kita melakukan beberapa hal berikut:

* Mengubah format gambar menjadi TensorImage.
* Mengubah ukuran gambar.
* Melakukan normalisasi.

Untuk melakukan tahap preprocessing, implementasikan kode berikut.

In [ ]:
private TensorImage loadImage(final Bitmap bitmap) {
        // Loads bitmap into a TensorImage.
        inputImageBuffer.load(bitmap);
 
        // Creates processor for the TensorImage.
        int cropSize = Math.min(bitmap.getWidth(), bitmap.getHeight());
        // TODO(b/143564309): Fuse ops inside ImageProcessor.
        ImageProcessor imageProcessor =
                new ImageProcessor.Builder()
                        .add(new ResizeWithCropOrPadOp(cropSize, cropSize))
                        .add(new ResizeOp(imageSizeX, imageSizeY, ResizeOp.ResizeMethod.NEAREST_NEIGHBOR))
                        .add(getPreprocessNormalizeOp())
                        .build();
        return imageProcessor.process(inputImageBuffer);
    }
 
    private TensorOperator getPreprocessNormalizeOp() {
        return new NormalizeOp(IMAGE_MEAN, IMAGE_STD);
    }

Setelah itu, kita perlu memanggil dan membaca model `vegs.tflite` dengan menambahkan fungsi berikut.

In [ ]:
private MappedByteBuffer loadmodelfile(Activity activity) throws IOException {
        String MODEL_NAME = "vegs.tflite";
        AssetFileDescriptor fileDescriptor = activity.getAssets().openFd(MODEL_NAME);
        FileInputStream inputStream = new FileInputStream(fileDescriptor.getFileDescriptor());
        FileChannel fileChannel = inputStream.getChannel();
        long startoffset = fileDescriptor.getStartOffset();
        long declaredLength = fileDescriptor.getDeclaredLength();
        return fileChannel.map(FileChannel.MapMode.READ_ONLY, startoffset, declaredLength);
    }

Tahap berikutnya adalah menambahkan fungsi untuk melakukan klasifikasi gambar. Output dari fungsi ini berupa nilai probabilitas sebuah gambar yang diklasifikasikan ke dalam kategori atau label tertentu.

In [ ]:
 void classifyImage(Bitmap bitmap){
        int imageTensorIndex = 0;
        int[] imageShape = tflite.getInputTensor(imageTensorIndex).shape(); // {1, height, width, 3}
        imageSizeY = imageShape[1];
        imageSizeX = imageShape[2];
        DataType imageDataType = tflite.getInputTensor(imageTensorIndex).dataType();
 
        int probabilityTensorIndex = 0;
        int[] probabilityShape =
                tflite.getOutputTensor(probabilityTensorIndex).shape(); // {1, NUM_CLASSES}
        DataType probabilityDataType = tflite.getOutputTensor(probabilityTensorIndex).dataType();
 
        inputImageBuffer = new TensorImage(imageDataType);
        outputProbabilityBuffer = TensorBuffer.createFixedSize(probabilityShape, probabilityDataType);
        probabilityProcessor = new TensorProcessor.Builder().add(getPostprocessNormalizeOp()).build();
 
        inputImageBuffer = loadImage(bitmap);
 
        tflite.run(inputImageBuffer.getBuffer(),outputProbabilityBuffer.getBuffer().rewind());
    }

Seperti yang telah dijelaskan sebelumnya, output dari fungsi di atas berupa nilai probabilitas. Sehingga, kita perlu melakukan tahap postprocessing terlebih dahulu untuk memilih label dengan probabilitas paling tinggi. Tahap postprocessing dilakukan menggunakan file `vegs.txt` sebagai acuan label.

In [ ]:
public String showresult() {
        try {
            labels = FileUtil.loadLabels(context, "vegs.txt");
        } catch (Exception e) {
            e.printStackTrace();
            return null;
        }
        Map<String, Float> labeledProbability =
                new TensorLabel(labels, probabilityProcessor.process(outputProbabilityBuffer))
                        .getMapWithFloatValue();
        float maxValueInMap = (Collections.max(labeledProbability.values()));
        String result = null;
        for (Map.Entry<String, Float> entry : labeledProbability.entrySet()) {
            if (entry.getValue() == maxValueInMap) {
                result = entry.getKey();
            }
        }
 
        return result;
    }
 
    private TensorOperator getPostprocessNormalizeOp() {
        return new NormalizeOp(PROBABILITY_MEAN, PROBABILITY_STD);
    }

Jika semua tahap telah dilakukan, berkas `TFLiteHelper.java` Anda akan terlihat seperti ini:

package com.example.imageclassification;

In [ ]:
import android.app.Activity;
import android.content.res.AssetFileDescriptor;
import android.graphics.Bitmap;
 
import org.tensorflow.lite.DataType;
import org.tensorflow.lite.Interpreter;
import org.tensorflow.lite.support.common.FileUtil;
import org.tensorflow.lite.support.common.TensorOperator;
import org.tensorflow.lite.support.common.TensorProcessor;
import org.tensorflow.lite.support.common.ops.NormalizeOp;
import org.tensorflow.lite.support.image.ImageProcessor;
import org.tensorflow.lite.support.image.TensorImage;
import org.tensorflow.lite.support.image.ops.ResizeOp;
import org.tensorflow.lite.support.image.ops.ResizeWithCropOrPadOp;
import org.tensorflow.lite.support.label.TensorLabel;
import org.tensorflow.lite.support.tensorbuffer.TensorBuffer;
 
import java.io.FileInputStream;
import java.io.IOException;
import java.nio.MappedByteBuffer;
import java.nio.channels.FileChannel;
import java.util.Collections;
import java.util.List;
import java.util.Map;
 
public class TFLiteHelper {
 
    private int imageSizeX;
    private int imageSizeY;
 
    private List<String> labels;
    private Interpreter tflite;
 
    private MappedByteBuffer tfliteModel;
    private TensorImage inputImageBuffer;
    private TensorBuffer outputProbabilityBuffer;
    private TensorProcessor probabilityProcessor;
 
    private static final float IMAGE_MEAN = 0.0f;
    private static final float IMAGE_STD = 1.0f;
 
    private static final float PROBABILITY_MEAN = 0.0f;
    private static final float PROBABILITY_STD = 255.0f;
 
    private Activity context;
 
    TFLiteHelper(Activity context) {
        this.context = context;
    }
 
    // ---- Kolom inisiasi TensorFlow Lite Interpreter ----
 
    void init() {
        try {
            Interpreter.Options opt = new Interpreter.Options();
            tflite = new Interpreter(loadmodelfile(context), opt);
        } catch (Exception e) {
            e.printStackTrace();
        }
    }
    // ----------------------------------------------------
 
    // ---- Kolom preprocessing gambar ----
    private TensorImage loadImage(final Bitmap bitmap) {
        // Loads bitmap into a TensorImage.
        inputImageBuffer.load(bitmap);
 
        // Creates processor for the TensorImage.
        int cropSize = Math.min(bitmap.getWidth(), bitmap.getHeight());
        // TODO(b/143564309): Fuse ops inside ImageProcessor.
        ImageProcessor imageProcessor =
                new ImageProcessor.Builder()
                        .add(new ResizeWithCropOrPadOp(cropSize, cropSize))
                        .add(new ResizeOp(imageSizeX, imageSizeY, ResizeOp.ResizeMethod.NEAREST_NEIGHBOR))
                        .add(getPreprocessNormalizeOp())
                        .build();
        return imageProcessor.process(inputImageBuffer);
    }
    // ----------------------------------------------------
 
    // ---- Kolom pemanggilan model tflite ----
    private MappedByteBuffer loadmodelfile(Activity activity) throws IOException {
        String MODEL_NAME = "vegs.tflite";
        AssetFileDescriptor fileDescriptor = activity.getAssets().openFd(MODEL_NAME);
        FileInputStream inputStream = new FileInputStream(fileDescriptor.getFileDescriptor());
        FileChannel fileChannel = inputStream.getChannel();
        long startoffset = fileDescriptor.getStartOffset();
        long declaredLength = fileDescriptor.getDeclaredLength();
        return fileChannel.map(FileChannel.MapMode.READ_ONLY, startoffset, declaredLength);
    }
 
    void classifyImage(Bitmap bitmap){
        int imageTensorIndex = 0;
        int[] imageShape = tflite.getInputTensor(imageTensorIndex).shape(); // {1, height, width, 3}
        imageSizeY = imageShape[1];
        imageSizeX = imageShape[2];
        DataType imageDataType = tflite.getInputTensor(imageTensorIndex).dataType();
 
        int probabilityTensorIndex = 0;
        int[] probabilityShape =
                tflite.getOutputTensor(probabilityTensorIndex).shape(); // {1, NUM_CLASSES}
        DataType probabilityDataType = tflite.getOutputTensor(probabilityTensorIndex).dataType();
 
        inputImageBuffer = new TensorImage(imageDataType);
        outputProbabilityBuffer = TensorBuffer.createFixedSize(probabilityShape, probabilityDataType);
        probabilityProcessor = new TensorProcessor.Builder().add(getPostprocessNormalizeOp()).build();
 
        inputImageBuffer = loadImage(bitmap);
 
        tflite.run(inputImageBuffer.getBuffer(),outputProbabilityBuffer.getBuffer().rewind());
 
    }
 
    private TensorOperator getPreprocessNormalizeOp() {
        return new NormalizeOp(IMAGE_MEAN, IMAGE_STD);
    }
    // ----------------------------------------------------
 
    // ---- Kolom postprocessing ----
    public String showresult() {
        try {
            labels = FileUtil.loadLabels(context, "vegs.txt");
        } catch (Exception e) {
            e.printStackTrace();
            return null;
        }
        Map<String, Float> labeledProbability =
                new TensorLabel(labels, probabilityProcessor.process(outputProbabilityBuffer))
                        .getMapWithFloatValue();
        float maxValueInMap = (Collections.max(labeledProbability.values()));
        String result = null;
        for (Map.Entry<String, Float> entry : labeledProbability.entrySet()) {
            if (entry.getValue() == maxValueInMap) {
                result = entry.getKey();
            }
        }
 
        return result;
    }
 
    private TensorOperator getPostprocessNormalizeOp() {
        return new NormalizeOp(PROBABILITY_MEAN, PROBABILITY_STD);
    }
    // ----------------------------------------------------
 
}

**Membuat file ‘MainActivity.java’**

Sebagai penutup, hubungkan UI dan `TFLiteHelper.java`. Caranya, bukalah berkas ‘MainActivity.java’ dan tambahkan perintah berikut:

In [ ]:
 ImageView imageView;
    Uri imageuri;
    Button buclassify;
    TextView classitext;
 
    TFLiteHelper tfLiteHelper;
    private Bitmap bitmap;
 
    @Override
    protected void onCreate(Bundle savedInstanceState) {
 
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);
        imageView = (ImageView) findViewById(R.id.image);
        buclassify = (Button) findViewById(R.id.classify);
        classitext = (TextView) findViewById(R.id.classifytext);
 
        tfLiteHelper = new TFLiteHelper(this);
        tfLiteHelper.init();
 
        imageView.setOnClickListener(selectImageListener);
        buclassify.setOnClickListener(classifyImageListtener);
 
    }
 
    @Override
    protected void onActivityResult(int requestCode, int resultCode, @Nullable Intent data) {
        super.onActivityResult(requestCode, resultCode, data);
        if (requestCode == 12 && resultCode == RESULT_OK && data != null) {
            imageuri = data.getData();
            try {
                bitmap = MediaStore.Images.Media.getBitmap(getContentResolver(), imageuri);
                imageView.setImageBitmap(bitmap);
            } catch (IOException e) {
                e.printStackTrace();
            }
        }
    }
 
    View.OnClickListener selectImageListener = new View.OnClickListener() {
        @Override
        public void onClick(View view) {
            String SELECT_TYPE = "image/*";
            String SELECT_PICTURE = "Select Picture";
 
            Intent intent = new Intent();
            intent.setType(SELECT_TYPE);
            intent.setAction(Intent.ACTION_GET_CONTENT);
            startActivityForResult(Intent.createChooser(intent, SELECT_PICTURE), 12);
        }
    };
 
    View.OnClickListener classifyImageListtener = new View.OnClickListener() {
        @Override
        public void onClick(View v) {
            if (bitmap != null) {
                tfLiteHelper.classifyImage(bitmap);
                setLabel(tfLiteHelper.showresult());
            }
        }
    };
 
    void setLabel(String label) {
        classitext.setText(label);
    }

Setelah menambahkan perintah di atas, hasil akhir dari berkas `MainActivity.java` akan terlihat seperti ini:

In [ ]:
package com.example.imageclassification;
 
import android.content.Intent;
import android.graphics.Bitmap;
import android.net.Uri;
import android.os.Bundle;
import android.provider.MediaStore;
import android.view.View;
import android.widget.Button;
import android.widget.ImageView;
import android.widget.TextView;
 
import androidx.annotation.Nullable;
import androidx.appcompat.app.AppCompatActivity;
 
import java.io.IOException;
 
public class MainActivity extends AppCompatActivity {
 
    // ---- Tambahkan contoh kodingan disini ----
    ImageView imageView;
    Uri imageuri;
    Button buclassify;
    TextView classitext;
 
    TFLiteHelper tfLiteHelper;
    private Bitmap bitmap;
 
    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        setContentView(R.layout.activity_main);
        imageView = (ImageView) findViewById(R.id.image);
        buclassify = (Button) findViewById(R.id.classify);
        classitext = (TextView) findViewById(R.id.classifytext);
 
        tfLiteHelper = new TFLiteHelper(this);
        tfLiteHelper.init();
 
        imageView.setOnClickListener(selectImageListener);
        buclassify.setOnClickListener(classifyImageListtener);
 
    }
 
    @Override
    protected void onActivityResult(int requestCode, int resultCode, @Nullable Intent data) {
        super.onActivityResult(requestCode, resultCode, data);
        if (requestCode == 12 && resultCode == RESULT_OK && data != null) {
            imageuri = data.getData();
            try {
                bitmap = MediaStore.Images.Media.getBitmap(getContentResolver(), imageuri);
                imageView.setImageBitmap(bitmap);
            } catch (IOException e) {
                e.printStackTrace();
            }
        }
    }
 
    View.OnClickListener selectImageListener = new View.OnClickListener() {
        @Override
        public void onClick(View view) {
            String SELECT_TYPE = "image/*";
            String SELECT_PICTURE = "Select Picture";
 
            Intent intent = new Intent();
            intent.setType(SELECT_TYPE);
            intent.setAction(Intent.ACTION_GET_CONTENT);
            startActivityForResult(Intent.createChooser(intent, SELECT_PICTURE), 12);
        }
    };
 
    View.OnClickListener classifyImageListtener = new View.OnClickListener() {
        @Override
        public void onClick(View v) {
            if (bitmap != null) {
                tfLiteHelper.classifyImage(bitmap);
                setLabel(tfLiteHelper.showresult());
            }
        }
    };
 
    void setLabel(String label) {
        classitext.setText(label);
    }
    // ----------------------------------------------------
}

# **Menguji Hasil Deployment**
Pada tahap ini Anda akan menguji hasil deployment dengan menjalankan model untuk mengklasifikasi gambar bahan makanan yang diinput oleh pengguna. Untuk menguji hasil deployment, jalankan aplikasi Android menggunakan Android Emulator yang tersedia pada Android Studio. Berikut merupakan tahapan untuk menjalankan aplikasi Android menggunakan Android Emulator:

1. Pastikan Anda telah membuat virtual device pada Android Studio.
2. Pilih virtual device yang Anda inginkan (pada latihan ini menggunakan “Pixel 5”).
3. Tekan tombol “Run app”.

Berikut merupakan gambar tahapan untuk menjalankan aplikasi Android menggunakan Android Emulator.

https://dicoding-web-img.sgp1.cdn.digitaloceanspaces.com/original/academy/dos:e4138aabe362399f3f8091cfb9fc984c20211208161830.jpeg

Jika semua tahapan berjalan dengan baik, akan muncul tampilan awal aplikasi Android sebagai berikut.

https://dicoding-web-img.sgp1.cdn.digitaloceanspaces.com/original/academy/dos:44f5cb57331dafbf4280953e5a8fc90120211208161853.png

Pada tampilan awal, Anda akan diminta untuk memilih gambar bahan makanan yang ingin diprediksi. Berikut langkah-langkah untuk menjalankan aplikasi yang sudah kita buat.

1. Tekan daerah berwarna hijau, Anda akan diarahkan pada direktori virtual device yang telah dibuat sebelumnya.
2. Drag file gambar yang diinginkan dari direktori lokal ke direktori virtual device.
3. Gambar tersebut akan secara otomatis masuk ke direktori “downloads” yang ada dalam virtual device.
4. Pilihlah gambar yang ada di direktori “downloads” tersebut.
5. Tekan tombol “CLASSIFY” untuk memprediksi jenis bahan makanan yang ada pada gambar.

Supaya lebih jelas, berikut alur dari langkah-langkah di atas dalam bentuk gambar.

https://dicoding-web-img.sgp1.cdn.digitaloceanspaces.com/original/academy/dos:7761856566b4079b4930b4da6192663920211208162017.jpeg

Ketika menekan tombol "CLASSIFY", aplikasi Android akan menampilkan output berupa prediksi kategori bahan masakan yang ada pada gambar. Hasil prediksi diambil dari kategori dengan nilai probabilitas paling tinggi.

https://dicoding-web-img.sgp1.cdn.digitaloceanspaces.com/original/academy/dos:5affa9eab00b95f31736a439baa8405020211208162043.jpeg

Selesai!

Sampai tahap ini, Anda telah berhasil membuat proyek untuk mengklasifikasi gambar beberapa jenis bahan makanan. Anda juga telah mampu menerapkan model pada aplikasi Android dengan TensorFlow Lite serta menguji hasil deployment. Well done!
